## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_data.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,East London,ZA,-33.0153,27.9116,54.54,100,100,16.11,mist
1,1,Kruisfontein,ZA,-34.0033,24.7314,51.89,74,88,17.07,light rain
2,2,Piney Green,US,34.7160,-77.3202,81.18,94,75,0.00,light rain
3,3,Kayes,ML,14.0000,-11.0000,84.02,70,72,8.77,light rain
4,4,Busselton,AU,-33.6500,115.3333,50.43,81,28,4.52,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 100


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
2,2,Piney Green,US,34.7160,-77.3202,81.18,94,75,0.00,light rain
3,3,Kayes,ML,14.0000,-11.0000,84.02,70,72,8.77,light rain
5,5,Madinat Sittah Uktubar,EG,29.8167,31.0500,89.20,30,0,14.65,clear sky
7,7,Rikitea,PF,-23.1203,-134.9692,71.92,82,85,15.66,overcast clouds
8,8,Portsmouth,GB,50.7990,-1.0913,75.00,68,98,8.01,overcast clouds
11,11,Albany,US,42.6001,-73.9662,85.68,67,70,3.00,broken clouds
14,14,Kapaa,US,22.0752,-159.3190,77.32,86,40,12.66,light rain
15,15,Bethel,US,41.3712,-73.4140,84.79,68,100,5.75,overcast clouds
16,16,Biltine,TD,14.5333,20.9167,90.09,41,86,2.71,overcast clouds
19,19,Butaritari,KI,3.0707,172.7902,81.54,75,98,14.81,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.isnull().values.any() == True:
    print("Null values are present in the preferred cities dataframe")

Null values are present in the preferred cities dataframe


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_clean_df = preferred_cities_df
preferred_cities_clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
2,2,Piney Green,US,34.7160,-77.3202,81.18,94,75,0.00,light rain
3,3,Kayes,ML,14.0000,-11.0000,84.02,70,72,8.77,light rain
5,5,Madinat Sittah Uktubar,EG,29.8167,31.0500,89.20,30,0,14.65,clear sky
7,7,Rikitea,PF,-23.1203,-134.9692,71.92,82,85,15.66,overcast clouds
8,8,Portsmouth,GB,50.7990,-1.0913,75.00,68,98,8.01,overcast clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Piney Green,US,81.18,light rain,34.7160,-77.3202,
3,Kayes,ML,84.02,light rain,14.0000,-11.0000,
5,Madinat Sittah Uktubar,EG,89.20,clear sky,29.8167,31.0500,
7,Rikitea,PF,71.92,overcast clouds,-23.1203,-134.9692,
8,Portsmouth,GB,75.00,overcast clouds,50.7990,-1.0913,
11,Albany,US,85.68,broken clouds,42.6001,-73.9662,
14,Kapaa,US,77.32,light rain,22.0752,-159.3190,
15,Bethel,US,84.79,overcast clouds,41.3712,-73.4140,
16,Biltine,TD,90.09,overcast clouds,14.5333,20.9167,
19,Butaritari,KI,81.54,overcast clouds,3.0707,172.7902,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
print("Done.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Piney Green,US,81.18,light rain,34.7160,-77.3202,Holiday City MHC
3,Kayes,ML,84.02,light rain,14.0000,-11.0000,
5,Madinat Sittah Uktubar,EG,89.20,clear sky,29.8167,31.0500,Happy Kennel Academy
7,Rikitea,PF,71.92,overcast clouds,-23.1203,-134.9692,People ThankYou
8,Portsmouth,GB,75.00,overcast clouds,50.7990,-1.0913,Royal Beach Hotel


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))